First create another container (Gold layer) on your adls and create external location on databricks 

You need to add a storage path to your catalog creation command — something like this:

In [0]:
# Create Catalog and Schema
spark.sql("""
CREATE CATALOG IF NOT EXISTS safe_sure_catalog
MANAGED LOCATION 'abfss://gold-layer@sampledemos.dfs.core.windows.net/'
""")

# Switch to the Catalog
spark.sql("USE CATALOG safe_sure_catalog")

# Create Schema (use either DATABASE or SCHEMA — they are synonyms in Databricks)
spark.sql("CREATE SCHEMA IF NOT EXISTS gold")
spark.sql("USE gold")


Create the Incident Delta table

In [0]:
%sql
CREATE TABLE IF NOT EXISTS gold.incident (
    IncidentID STRING,
    ClaimID STRING,
    IncidentDate DATE,
    IncidentType STRING,
    Severity STRING,
    Description STRING
)
USING DELTA;


In [0]:
# load the data from the silver layer 

df = spark.read.option("header", "true").csv("abfss://landingzone@sampledemos.dfs.core.windows.net/Incident.csv")


from pyspark.sql import functions as F


# Data Cleaning
df_clean = (df
    .withColumn("IncidentDate", F.to_date("IncidentDate", "yyyy/MM/dd"))
    .withColumn("IncidentType", F.trim(F.initcap("IncidentType")))
    .withColumn("Severity", F.upper(F.trim("Severity")))
)

# Save cleaned data to Delta table
df_clean.write.mode("overwrite").saveAsTable("gold.incident")

In [0]:
%sql
select * from gold.incident